In [ ]:
# default_exp hit_test
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# HitTest

> VAULT Proposal python module for addressing Technical Scenario objective 1

Objective 1:
> 1) Determine the “hits” where a satellite has geodetic overlap of any vessel(s) at any point(s) in time. For simplicity, it may be assumed that a satellite has full view of half the earth (regardless of satellite type or its elevation above the earth). However, additional accuracy models with rationale is allowed.

A satellite can see the ship only if the ship can see the satellite. So instead of the half-earth assumption, we define a "hit" as the satellite being above the horizon (defined as "alt" ≥0º in a ship-based alt-azimuth coordinate system). 

According to [exactEarth](http://www.marinetraffic.org/exactearth/), a more realistic approximation is that AIS satellites at 650 km have a 5000 km diameter Field of View. That corresponds to a horizon of about 𝜃=14.6º, as shown here:

                             🛰
                             : `
                             :     `
                         650 :         ` 
                          km :             `
                             :                 `
                             :    2500km        𝜃  `
                             .........................`🛳

In [ ]:
import math
𝜃 = math.degrees(math.atan(650/2500))
print(f'{𝜃:.1f}')

14.6


In [ ]:
# export

# Requires modules in ../jacobs_vault be available. Either:
#    ln -s nbs/jacobs_vault -> jacobs_vault
# or:
#    in each notebook `import sys; sys.path.append('..')`
# or:
#    add .. to PYTHONPATH.

from jacobs_vault.starmap import starmap  # Plotting fn

from datetime import datetime
from dateutil import tz
from skyfield.api import EarthSatellite
from skyfield.api import Topos, load
import math
import pandas as pd
import numpy as np

In [ ]:
# hide
#from nbdev.showdoc import *

### Define Schema and Path

Save memory by using appropriate data types for the TLE elements. (Determined by peeking at the data file or reading the TLE format.) This saves quite a bit, and may save more as Pandas optimizes the `string` class vs. default `object`. Regardless, it makes columns single-typed.

In [ ]:
#export
COLUMNS = ["satellite", "day_dt", "day", "tle_dt", "tle_ts", "line1", "line2"]
# DTYPES = [str, str, int, str, int, str, str]
DTYPES = {'satellite': 'uint16', # observed values are ints in 5..41678, so 0..65535 is good
          'day_dt': 'str',       # here a single date, but generally datetime: PARSE
          'day': 'uint16',       # here a single value 6026, too big for uint8, but 16 is good
          'tle_dt': 'str',       # again, PARSE AS DATETIME
          'tle_ts': 'uint32',    # large ints, but < 4294967295. We could compress more, but... meh
          'line1': 'string',     # 12K unique 80-char TLE strings. Category wd give tiny compression.
          'line2': 'string'}     # In theory "string" is better than "object". Not seeing it here.

DATE_COLS = ['day_dt', 'tle_dt']

# Where to look for the TLE dayfiles.
# Symlink ../data to the actual data.
DAY_FILE_PATH="../data/VAULT_Data/TLE_daily"


### Quality Values


In [ ]:
# export

# Set horizon in degrees. Suggested: 0º or 14.6º.
HORIZON = 14.6

# Define cutoffs for TLE track quality, as TLE age in days
EXCELLENT, GOOD, POOR = 2, 14, 56

def get_qvals(𝚫t: int, alt: float, 𝜃:float=HORIZON):
    """Get quality vals for raw 𝚫t [days].
    
    Returns Series (alt, az, 𝚫t) in units (º,º, days)
    
    Params
    ------
        𝚫t - age of TLE in days. Int or flot.
        alt - altitude above horizon
        𝜃 - minimum alt in degrees to count as a hit (Default HORIZON)
        
    """

    if 𝚫t <= EXCELLENT:
        if alt.degrees > 𝜃:
            qvals = ["Excellent", math.nan]
        else:
            qvals = [math.nan, "Excellent"]
    elif 𝚫t <= GOOD:
        if alt.degrees > 0.0:
            qvals = ["Good", math.nan]
        else:
            qvals = [math.nan, "Good"]
    elif 𝚫t <= POOR:
        if alt.degrees > 0.0:
            qvals = ["Poor", math.nan]
        else:
            qvals = [math.nan, "Poor"]
    else:
        qvals = [math.nan, "Stale"]

    return qvals

#

### HitTest Class

During data posturing we created single-day TLE files, so each day contains the most recent TLE for all 12K satellites.

The `HitTest` constructor takes a date, loads the TLE dayfile, and returns the corresponding DataFrame.

_TODO_: Most of these satellites are useless. Speed by ignoring.

In [ ]:
#export

class HitTest:
    """ Counts the satellites that are visible at a given point on the globe at a
    given time, and returns counts classified by data quality and
    latitude, azimuth, hit_quality, radius for visible satellites
    """
    def __init__(self, dt:datetime, 
                 day_file_base_path:str=DAY_FILE_PATH, 
                 𝜃:float=HORIZON):
        """Look for and load TLE datafile for date {dt}."""
        
        df_path = "%s/%4d/%02d/%02d.tab.gz" % (day_file_base_path, dt.year, dt.month, dt.day)
        print(f"Trying to load {df_path}")
        df = pd.read_csv(df_path,
                         names=COLUMNS, sep='\t', compression='gzip',
                         dtype=DTYPES,
                         parse_dates=DATE_COLS,
                         infer_datetime_format=True)
        self.df_day_tle = df.drop_duplicates()
        self.𝜃 = 𝜃
        self.dt = dt
    #


    def satellite_alt_az_days(self, _t0: datetime, lat: float, lon: float):
        '''Load tracks for day {_t0} and return altº, azº, and 𝚫t [days]
        for each row.
        
        Usage eg: satellite_alt_az_days(datetime(2016, 6, 30), 45.0, -176.0)

        '''
        earth_position = Topos(lat, lon)

        ts = load.timescale()
        t = ts.utc(_t0.replace(tzinfo=tz.tzutc()))

        def eval_tle(row):
            '''Extract satellite info from line1/line2/tle_dt.

            Returns alt, az, and (days between dt and each row).
            Inherits {ts}, {t}, and {earth_position} values at function definition.

            TODO: Currently only works for `apply(raw=False)`.

            '''
            try:
                satellite = EarthSatellite(row['line1'], row['line2'], 'x', ts)
                𝚫t = abs(_t0 - row['tle_dt']).days
            except IndexError:
                # `apply(raw=True)` sends arrays instead of Series
                satellite = EarthSatellite(row[5], row[6], 'x', ts)
                𝚫t = abs(_t0 - row[3]).days
            topocentric = (satellite - earth_position).at(t)
            alt, az, distance = topocentric.altaz()
            qvals = get_qvals(𝚫t, alt)
            return pd.Series([alt.degrees, az.degrees, 𝚫t] + qvals)

        _ = self.df_day_tle.apply(eval_tle, axis=1, raw=False)
        df_alt_az_days = pd.DataFrame(_)
        df_alt_az_days.columns = ["altitude", "azimuth", "days", "hit", "miss"]
        #df_alt_az_days.reindex()
        return df_alt_az_days


    def invoke(self, dt: datetime, lat: float, lon: float):
        ''' Main logic for satellite hit-testing service

            Returns 2 DataFrames:
             - df_hit_miss_table :       The hit,miss stats table
             - df_alt_az_days_visible :  The information on the visible satellites for star-map plotting
             
        '''
        df_alt_az_days = self.satellite_alt_az_days(dt, lat, lon)

        # "invert" altitude for polar plotting.  Doing this thousands of times
        #  more than necessary (really just want R for the df_alt_az_days_visible slice)
        #  but pandas does not like apply on a slice.
        df_alt_az_days.loc["R"] = 90.0 - df_alt_az_days["altitude"]

        def apply_quality_str(row, col):
            q = ""
            if row[col] == Q_EXCELLENT:
                q = "Excellent"
            elif row[col] == Q_GOOD:
                q = "Good"
            elif row[col] == Q_POOR:
                q = "Poor"
            elif row[col] == Q_STALE:
                q = "Stale"
            # no-else ... leave the NaNs alone
            return q
        #

        df_hit_miss_table = pd.concat([
            df_alt_az_days["hit"].value_counts(sort=False),
            df_alt_az_days["miss"].value_counts()], 
            axis=1, sort=False)

        df_alt_az_days_visible = df_alt_az_days[df_alt_az_days["altitude"] > self.𝜃]

        return df_hit_miss_table, df_alt_az_days_visible
    #

    def web_invoke(self, dt, lat, lon):
        ''' Main support function for satellite hit-testing service

            returns a json object having two objects:
            {
                "hitmiss": The hit,miss stats table
                "visible": The information on the visible satellites
            }
        '''
        df_hit_miss_table, df_alt_az_days_visible = self.invoke(dt, lat, lon)
        result = {
            "hitmiss": df_hit_miss_table.to_dict(),
            "visible": df_alt_az_days_visible.to_dict()
        }
        return json.dumps(result)
    #

### Execute for a given day

2016-06-30 for starters

In [ ]:
dt = datetime(2016, 6, 30)
ht = HitTest(dt)
hitmiss, rows = ht.invoke(dt, 45.0, -176.0)

Trying to load ../data/VAULT_Data/TLE_daily/2016/06/30.tab.gz


In [ ]:
hitmiss

,hit,miss
Excellent,799.0,11130
Poor,1.0,9
Good,30.0,178
Stale,NaN,5


In [ ]:
rows.sample(5)

,altitude,azimuth,days,hit,miss
4792,39.103415,187.715195,0.0,Excellent,NaN
11229,40.888612,165.186133,0.0,Excellent,NaN
12587,86.544701,192.259231,0.0,Excellent,NaN
3920,45.862176,134.587456,0.0,Excellent,NaN
14511,51.141112,6.748791,0.0,Excellent,NaN


## Visualize the results

Generate a polar alt/az plot of the qualifying satellites
* Excellent = blue
* Good = red
* Else = yellow


**TODO:** Is "0" here 0 altitude? That would be on the horizon, which is counter-intuitive. 

Note the band of satellites at southern bearings -- this ship was in the Northern hemisphere. 

In [ ]:
fig = starmap(rows)

In [ ]:
fig.write_image('images/starmap_new.pdf')